In [2]:
import pandas as pd
import re
from transformers import T5Tokenizer,AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

In [3]:
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")


df = pd.read_csv("../data/hiyoritalk.csv", index_col=0).reset_index()

texts = []


for i in range(len(df)):

    text = df.iloc[i, 3]

    if type(text) != str:
        continue

    text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", text)
    if text == "":
        continue
    # if text doesn't end with "\n", add "\n" 
    if text[-1] != "\n":
        text += "\n"

    
    texts.append(text)


train_data_path = "train.text"
with open(train_data_path, "w") as f:
    f.writelines(texts)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
train_dataset = TextDataset(
    tokenizer = tokenizer,
    file_path = train_data_path,
    block_size = 256 #文章の長さを揃える必要がある
)

#データの入力に関する設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 訓練に関する設定
training_args = TrainingArguments(
    output_dir="./gpt2-ft",  # 関連ファイルを保存するパス
    overwrite_output_dir=True,  # ファイルを上書きするかどうか
    num_train_epochs=3,  # エポック数
    per_device_train_batch_size=8,  # バッチサイズ
    logging_steps=100,  # 途中経過を表示する間隔
    save_steps=800  # モデルを保存する間隔
)

#トレーナーの設定
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator = data_collator,
    train_dataset = train_dataset
)

/Users/tada/Library/Caches/pypoetry/virtualenvs/hamahiyo-eXwOOx2--py3.12/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/Users/tada/Library/Caches/pypoetry/virtualenvs/hamahiyo-eXwOOx2--py3.12/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=Tru

In [5]:
%%time
trainer.train()

  0%|          | 0/93 [00:00<?, ?it/s]

KeyboardInterrupt: 